### Geodesic arcs between points ###
by: Richard Sowers
* <r-sowers@illinois.edu>
* <https://publish.illinois.edu/r-sowers/>

Copyright 2018 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license

### Explanation###
This code computes the value of pi.  More generally, tell the reader what the point of the code is

imports

In [1]:
import geographiclib.geodesic
import pandas
import numpy
geod = geographiclib.geodesic.Geodesic.WGS84
import json
import urllib.request
import urllib.parse

In [2]:
fname="destination_query_count"
APIKey=

In [3]:
def get_latlng(place):
    query="https://maps.googleapis.com/maps/api/geocode/json?address={0:s}&key={1:s}"
    
    def to_query(place):
        return query.format(urllib.parse.quote_plus(place),APIKey)
    
    
    with urllib.request.urlopen(to_query(place)) as response:
        place_json = json.loads(response.read())
        
    return place_json["results"][0]["geometry"]["location"]

    
get_latlng("Carnegie Mellon University")


{'lat': 40.4432027, 'lng': -79.9428499}

UIUC location

In [4]:
(UIUC_lat,UIUC_lon)=(40.1020,-88.2272)

Compute geodesic arc from UIUC to other point

In [5]:
def makearc(other_lat,other_lon,num=20):
    geoline = geod.InverseLine(UIUC_lat,UIUC_lon,other_lat,other_lon ,geographiclib.geodesic.Geodesic.LATITUDE | geographiclib.geodesic.Geodesic.LONGITUDE)
    waypoints=numpy.linspace(start=0,stop=geoline.a13,num=num)
    
    def getlatlng(waypoint):
        g = geoline.ArcPosition(waypoint, geographiclib.geodesic.Geodesic.LATITUDE |geographiclib.geodesic.Geodesic.LONGITUDE | geographiclib.geodesic.Geodesic.LONG_UNROLL)
        return [g["lon2"],g["lat2"]]
    
    out=[getlatlng(waypoint) for waypoint in waypoints]
    return out

In [6]:
def getfulldata(place):
    location=get_latlng(place)
    return makearc(location["lat"],location["lng"],num=100)
getfulldata("Columbia University)")

[[-88.2272, 40.102000000000004],
 [-88.08519806907594, 40.11787481702975],
 [-87.94313031251552, 40.13357542117023],
 [-87.80099739074848, 40.149101636272135],
 [-87.6587999670668, 40.16445328793198],
 [-87.51653870760077, 40.17963020350071],
 [-87.37421428129487, 40.194632212092394],
 [-87.23182735988316, 40.209459144592785],
 [-87.08937861786463, 40.224110833667744],
 [-86.94686873247807, 40.23858711377175],
 [-86.80429838367685, 40.252887821156165],
 [-86.66166825410325, 40.26701279387757],
 [-86.51897902906266, 40.28096187180593],
 [-86.37623139649739, 40.294734896632725],
 [-86.23342604696035, 40.308331711878964],
 [-86.09056367358836, 40.321752162903245],
 [-85.94764497207522, 40.33499609690954],
 [-85.80467064064453, 40.34806336295509],
 [-85.66164138002227, 40.360953811958055],
 [-85.51855789340907, 40.37366729670526],
 [-85.37542088645222, 40.38620367185968],
 [-85.23223106721758, 40.398562793968],
 [-85.08898914616107, 40.410744521467926],
 [-84.94569583609984, 40.42274871469

In [7]:
fullnames={
    "Berkeley":"University of California Berkeley",
    "Boston University":"Boston University",
    "Brandeis":"Brandeis Univeristy",
    "Chicago":"University of Chicago",
    "CMU":"Carnegie Mellon University",
    "Columbia":"Columbia University",
    "Cornell":"Cornell University",
    "Dartmouth":"Dartmouth University",
    "Georgia Tech":"Georgia Tech",
    "Harvard":"Harvard University",
    "NortheasternSeattle":"University of Northeastern at Seattle",
    "Northwestern":"Northwestern University",
    "NYU":"New York University",
    "Princeton":"Princeton University",
    "Purdue":"Purdue University",
    "South Carolina":"University of South Carolina",
    "Stanford":"Stanford University",
    "Toronto":"University of Toronto",
    "UCLA":"University of California at Los Angeles",
    "UIUC":"University of Illinois at Urbana-Champaign",
    "USC":"University of Southern California",
    "UCSD":"University of California San Diego",
    "Vanderbilt":"Vanderbilt University",
    "Yale":"Yale University"
}

In [8]:
paths={place:getfulldata(fullname) for (place,fullname) in fullnames.items()}

In [9]:
data_raw=pandas.read_excel(fname+".xlsx")
data=data_raw.copy()

In [10]:
out={record["university"]:{"path":paths[record["university"]],"count":record["count"]} for _,record in data.iterrows()}

In [11]:
with open('index.js','w') as f:
    print('var data='+json.dumps(out),file=f)